test

In [4]:
import cv2
import numpy as np


def stitch_images(img1, img2):
    """
    将两张图片进行拼接
    Args:
        img1: 第一张输入图片
        img2: 第二张输入图片
    Returns:
        result: 拼接后的图片
    """
    # 1. 特征点检测与匹配
    # 创建SIFT检测器
    sift = cv2.SIFT_create()
    
    # 检测特征点和计算描述符
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
    
    # 创建特征匹配器
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # 2. 应用比率测试筛选好的匹配点
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    
    # 3. 查找单应性矩阵
    if len(good_matches) >= 4:
        # 获取匹配点的坐标
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # 计算单应性矩阵
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        
        # 4. 图像变换和拼接
        # 获取图像尺寸
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        
        # 计算变换后图像的范围
        pts1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
        pts2 = cv2.perspectiveTransform(pts1, H)
        
        # 计算输出图像的尺寸和偏移
        pts = np.concatenate((pts2, np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)))
        [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
        t = [-xmin, -ymin]
        
        # 创建平移矩阵
        Ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]])
        
        # 对第一张图片进行变换
        result = cv2.warpPerspective(img1, Ht.dot(H), (xmax-xmin, ymax-ymin))
        
        # 5. 图像融合
        # 将第二张图片复制到结果图像中
        result[t[1]:h2+t[1], t[0]:w2+t[0]] = img2
        
        # 6. 处理重叠区域 - 使用加权融合
        mask1 = np.zeros((ymax-ymin, xmax-xmin), dtype=np.float32)
        mask1[t[1]:h2+t[1], t[0]:w2+t[0]] = 1
        mask2 = cv2.warpPerspective(np.ones_like(img1, dtype=np.float32), Ht.dot(H), (xmax-xmin, ymax-ymin))
        
        # 创建权重矩阵
        weight1 = cv2.GaussianBlur(mask1, (21, 21), 11)
        weight2 = cv2.GaussianBlur(mask2, (21, 21), 11)
        
        # 归一化权重
        weight1 = weight1 / (weight1 + weight2 + 1e-6)
        weight2 = weight2 / (weight1 + weight2 + 1e-6)
        
        # 加权融合
        warped_img1 = cv2.warpPerspective(img1, Ht.dot(H), (xmax-xmin, ymax-ymin))
        result = weight1[..., None] * result + weight2[..., None] * warped_img1
        
        return result.astype(np.uint8)
    
    return None

def main():
    """
    主函数，用于测试图像拼接
    """
    # 读取输入图像
    img1 = cv2.imread('image1.jpg')
    img2 = cv2.imread('image2.jpg')
    
    if img1 is None or img2 is None:
        print("Error: Could not read input images")
        return
    
    # 进行图像拼接
    result = stitch_images(img1, img2)
    
    if result is not None:
        # 保存结果
        cv2.imwrite('stitched_image.jpg', result)
        print("Stitching completed successfully!")
    else:
        print("Error: Could not stitch images")

if __name__ == "__main__":
    main()

ValueError: operands could not be broadcast together with shapes (2062,2069) (2062,2069,3) 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
